1. Processar Dados para Extrair Padrões

a) Crimes por Horário


In [ ]:
import pandas as pd

df = pd.read_csv('crimes_asa_sul_2020_2025_com_pessoas_endereco2.csv')

# Contar crimes por hora
df['hora'] = pd.to_datetime(df['hora']).dt.hour
crimes_por_hora = df.groupby('hora').size().reset_index(name='contagem')
crimes_por_hora = crimes_por_hora.sort_values(by='contagem', ascending=False)
print("Horários com mais crimes:")
print(crimes_por_hora.head(3))

C:\Users\migue\AppData\Local\Temp\ipykernel_27404\1691409007.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['hora'] = pd.to_datetime(df['hora']).dt.hour


Horários com mais crimes:
    hora  contagem
2      2       682
11    11       648
6      6       647


b) Crimes por Localização


In [22]:
# Contar crimes por rua
crimes_por_rua = df.groupby('rua').size().reset_index(name='contagem')
crimes_por_rua = crimes_por_rua.sort_values(by='contagem', ascending=False)
print("Ruas com mais crimes:")
print(crimes_por_rua.head(3))

Ruas com mais crimes:
       rua  contagem
1  CLS 402      1739
5  SQS 107      1730
3  SQS 105      1719


    c) Crimes por Tipo


In [23]:
# Contar crimes por tipo
crimes_por_tipo = df.groupby('tipo_crime').size().reset_index(name='contagem')
crimes_por_tipo = crimes_por_tipo.sort_values(by='contagem', ascending=False)
print("Tipos de crime mais comuns:")
print(crimes_por_tipo.head(3))

Tipos de crime mais comuns:
   tipo_crime  contagem
1       furto      4311
3       roubo      3323
5  vandalismo      2688


2. Criar um Modelo de Priorização Simples


a) Calcular Risco por Região


In [24]:
# Definir pesos por tipo de crime
pesos = {
    'feminicídio': 5,
    'homicídio': 5,
    'tráfico': 4,
    'roubo': 3,
    'furto': 2,
    'vandalismo': 1
}

# Adicionar coluna de peso
df['peso'] = df['tipo_crime'].map(pesos)

# Calcular pontuação de risco por rua
risco_por_rua = df.groupby('rua')['peso'].sum().reset_index(name='risco_total')
risco_por_rua = risco_por_rua.sort_values(by='risco_total', ascending=False)
print("Ruas com maior risco:")
print(risco_por_rua.head(3))

Ruas com maior risco:
       rua  risco_total
1  CLS 402       4527.0
3  SQS 105       4526.0
5  SQS 107       4414.0


b) Identificar Horários de Risco


In [25]:
# Filtrar crimes graves (ex: homicídio, tráfico)
crimes_graves = df[df['tipo_crime'].isin(['homicídio', 'tráfico'])]

# Contar crimes graves por hora
horarios_risco = crimes_graves.groupby('hora').size().reset_index(name='contagem')
horarios_risco = horarios_risco.sort_values(by='contagem', ascending=False)
print("Horários com mais crimes graves:")
print(horarios_risco.head(3))

Horários com mais crimes graves:
   hora  contagem
4   4.0       148
8   8.0       148
9   9.0       145


3. Visualizar os Resultados


In [45]:
import folium
import pandas as pd
import numpy as np

# Carregar dados
df = pd.read_csv('crimes_asa_sul_2020_2025_com_pessoas_endereco.csv')

# Calcular risco por rua
risco_por_rua = df.groupby('rua').agg(
    total_crimes=('tipo_crime', 'count'),
    risco_total=('peso', 'sum')
).reset_index()

# Calcular média de coordenadas por rua
rua_coords = df.groupby('rua')[['latitude', 'longitude']].mean().reset_index()

# Combinar risco com coordenadas
risco_com_coords = pd.merge(risco_por_rua, rua_coords, on='rua', how='left')

# Criar mapa centrado na Asa Sul
mapa = folium.Map(location=[-15.7942, -47.8825], zoom_start=15)

# Adicionar marcadores com validação
for _, row in risco_com_coords.sort_values(by='risco_total', ascending=False).head(5).iterrows():
    lat = row['latitude']
    lon = row['longitude']
    
    # Verificar se as coordenadas são válidas
    if pd.notna(lat) and pd.notna(lon):
        folium.Marker(
            location=[lat, lon],
            popup=f"{row['rua']} (Risco: {row['risco_total']})",
            icon=folium.Icon(color='red', icon='warning')
        ).add_to(mapa)
    else:
        print(f"⚠️ Coordenadas inválidas para {row['rua']}")

# Salvar o mapa
mapa.save('mapa_risco.html')
print("✅ Mapa salvo como 'mapa_risco.html'")

KeyError: "Column(s) ['peso'] do not exist"

In [35]:
import folium
import pandas as pd
from folium.plugins import HeatMap, MarkerCluster

# Carregar os dados (amostra de 10% para teste)
df = pd.read_csv('crimes_asa_sul_2020_2025_com_pessoas_endereco.csv').sample(n=1500, random_state=42)

# Criar mapa centrado na Asa Sul
mapa = folium.Map(location=[-15.7942, -47.8825], zoom_start=14, tiles='CartoDB positron')

# Adicionar marcadores com clusterização
marker_cluster = MarkerCluster().add_to(mapa)
for _, row in df.iterrows():
    folium.Marker(
        location=[row['latitude'], row['longitude']],
        popup=f"{row['rua']} - {row['tipo_crime']}",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(marker_cluster)

# Gerar heatmap com amostra menor (10% dos dados)
heat_data = [[row['latitude'], row['longitude']] for _, row in df.sample(n=500).iterrows()]
HeatMap(heat_data, radius=15, blur=20, max_zoom=16).add_to(mapa)

# Salvar o mapa
mapa.save('mapa_interativo_asa_sul.html')
print("✅ Mapa salvo como 'mapa_interativo_asa_sul.html'")

✅ Mapa salvo como 'mapa_interativo_asa_sul.html'


5. Exibir Rotas no Waze ou Google Maps


In [28]:
# Exemplo de URL para abrir no Waze
lat_destino = -15.7932
lon_destino = -47.8815
url_waze = f"https://waze.com/ul?ll={lat_destino}%2C{lon_destino}&navigate=yes"
print("🔗 Rota no Waze:", url_waze)

🔗 Rota no Waze: https://waze.com/ul?ll=-15.7932%2C-47.8815&navigate=yes
